In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import numpy as np
import pandas as pd
import math
import pickle
import time
import random
random.seed(1)

# This notebook creates question-answer mapping dictionaries used for all models.





# Question-Answer Mappings

Dictionary:
Key = each unique question_id  
Value = list of 10 tuples    ('answer_id', 1s or 0s labels) where 1 = accepted answer and 0 = not accepted answer

x4 Dictionaries will be created:
All / Train / Dev / Test



### Import Data

In [9]:
df_all = pd.read_csv (r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/raw_data_preprocessed.csv')
df_train = pd.read_csv (r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/train_dev_test_split_data/master_train_data.csv')
df_dev = pd.read_csv (r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/train_dev_test_split_data/master_dev_data.csv')
df_test = pd.read_csv (r'/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/train_dev_test_split_data/master_test_data.csv')
print("All Shape:", print(df_all.shape))
print("Train Shape:", print(df_train.shape))
print("Dev Shape:", print(df_dev.shape))
print("Test Shape:", print(df_test.shape))

print("Columns:", df_train.columns)
df_train.head(5)

(60336, 19)
All Shape: None
(48309, 19)
Train Shape: None
(6047, 19)
Dev Shape: None
(5980, 19)
Test Shape: None
Columns: Index(['question_id', 'question_title_raw', 'question_body_raw',
       'question_body', 'question_body_text', 'question_body_code',
       'question_title_body_raw', 'question_title_body',
       'question_title_body_text', 'question_title_body_code', 'answer_id',
       'answer_body_raw', 'answer_body', 'answer_text', 'answer_code',
       'answer_count', 'view_count', 'answer_score', 'is_accepted_answer'],
      dtype='object')


,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer
0,55661532,"When I run my socket program, there get: `impo...",<p>I write a socket server programmer:</p>\n\n...,I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,"When I run my socket program, there get: `impo...",I write a socket server programmer:\n#-*- codi...,I write a socket server programmer:\n\nwhen I ...,#-*- coding:utf-8 -*-\n# Author:sele\n\nimport...,55661630,<p>The answer to your new question is that you...,The answer to your new question is that you ca...,The answer to your new question is that you ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,1816,0,0
1,55661626,"with socket.socket(socket.AF_INET, socket.SOCK...",<p>I have a socket server program:</p>\n\n<pre...,I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,"with socket.socket(socket.AF_INET, socket.SOCK...",I have a socket server program:\n#!/usr/bin/en...,I have a socket server program:\n\nwhen I run ...,#!/usr/bin/env python\n#-*- coding:utf-8 -*-\n...,55661660,<p>You can't use <code>socket.socket(socket.AF...,"You can't use socket.socket(socket.AF_INET, so...",You can't use with . So that a statement ca...,"socket.socket(socket.AF_INET, socket.SOCK_STRE...",2,2521,0,1
2,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,55662821,<p>Define list as <code>[]</code> and use <cod...,Define list as [] and use append to add elemen...,Define list as and use to add elements to li...,"[]appendscript = []\n\nscript.append(['A','A',...",2,24,0,1
3,55662778,create list with spectial type - Spyder error ...,<p>I want to print <code>&gt;</code> in the sc...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,create list with spectial type - Spyder error ...,I want to print > in the script 0 to test the ...,I want to print in the script 0 to test the c...,>script[0][4][2]name 'script' is not defined\n...,55662828,<p>I modified your code slightly:</p>\n\n<pre>...,I modified your code slightly:\nscript = [['A'...,I modified your code slightly:\n\nand it works...,"script = [['A','A',['A'],['A'],['A','5','>'],'...",2,24,0,0
4,39746223,sys.stdin.readline() with if else (Python),<p>I am new at Python and need a little help. ...,I am new at Python and need a little help. \nI...,I am new at Python and need a little help. \nI...,"print(""What is your name ?"")\n\nname = sys.std...",sys.stdin.readline() with if else (Python)<p>I...,I am new at Python and need a little help. \nI...,I am new at Python and need a little help. \nI...,"print(""What is your name ?"")\n\nname = sys.std...",55669306,<p>In reference to the main post:</p>\n\n<p>I ...,In reference to the main post:\nI used sys.std...,In reference to the main post:\nI used sys.std...,import sys\nprint('how much was your meal')\np...,2,889,0,0


### Create array of unique question_id

In [10]:
unique_question_ids_all = np.array(df_all['question_id'].unique())   
unique_question_ids_train = np.array(df_train['question_id'].unique())   
unique_question_ids_dev = np.array(df_dev['question_id'].unique())   
unique_question_ids_test = np.array(df_test['question_id'].unique())   

print("All Length:", len(unique_question_ids_all))
print("Train Length:", len(unique_question_ids_train))
print("Dev Length:", len(unique_question_ids_dev))
print("Test Length:", len(unique_question_ids_test))
print(unique_question_ids_train)

All Length: 30623
Train Length: 24498
Dev Length: 3062
Test Length: 3063
[55661532 55661626 55662778 ... 49967909 50077084 50036470]


### Function to create question_answer_ids dictionaries

In [11]:
def q_a_dictionary(df, unique_question_id, num_questions, num_answers):

  """
  Function inputs dataframe and outputs a dictionary 
    Dictionary: Key = each unique question_id
    Value = list of 10 tuples ('answer_id', 1s or 0s labels) where 1 = accepted answer and 0 = not accepted answer
  
  Variables:
  df = dataframe 
  num_questions = value for number of questions in dictionary
  num_answers = value for number of answers per question

  """

  q_a_dictionary=dict()
  start=time.time()
  #Loop through unique question id's
  for q_id in unique_question_id[0:num_questions]:
    answer_ids_list = []  #list of tuples (answer_id, accepted answer 1's or 0's)

    #Append (answer_id, is_accepted_answer) tuples to answer_ids_list
    #These are the answers for each unique question 
    for each_index in df[df.question_id == q_id].index:
      answer_id = df.loc[df.index == each_index].answer_id.values[0]  
      is_accepted_answer = df.is_accepted_answer[each_index]       
      answer_ids_list.append( (answer_id,is_accepted_answer) )
  
    answer_ids_only = [a_tuple[0] for a_tuple in answer_ids_list]  # Random sampling (later) will ignore these answer_ids to avoid duplicates

    #Loop through random answers and add to answer_ids_list    
    while len(answer_ids_list) < num_answers:
      random_answer_index = random.sample(df.index.tolist(),1)       #Generate random answer_ids to add to list (negative sampling)
      if random_answer_index[0] not in answer_ids_only:
        answer_id = df.loc[df.index == random_answer_index[0]].answer_id.values[0]         
        answer_ids_list.append( (answer_id, 0) )  #0 because these random answers are not the accepted answers       

    #Error checking. There should be 10 answers and only 1 accepted answer
    if len(answer_ids_list) != 10: 
      print("ERROR: THERE ARE LESS THAN OR MORE THAN 10 ANSWERS")
      print("q_id:", q_id, "answer_ids_list: ", answer_ids_list)
      break
    elif sum([x[1] for x in answer_ids_list]) != 1:
      print("ERROR: THERE ARE LESS THAN OR MORE THAN 1 ACCEPTED ANSWER")
      print("q_id:", q_id, "answer_ids_list: ", answer_ids_list)
      break      
    else:
      #Add to dictionary
      q_a_dictionary[q_id] = answer_ids_list
    
  end=time.time()
  print("Time: %0.2f seconds" % (end-start) )
  return q_a_dictionary

### Creating train/dev/test question_answer_ids dictionaries

In [12]:
import time
start=time.time()

# Initiating Variables
num_answers=10      #number of answers per question

num_questions_all= len(unique_question_ids_all)
num_questions_train= len(unique_question_ids_train)
num_questions_dev= len(unique_question_ids_dev)
num_questions_test= len(unique_question_ids_test)

# Running Functions
q_a_dictionary_all   = q_a_dictionary(df_all,   unique_question_ids_all,   num_questions_all,   num_answers)
q_a_dictionary_train = q_a_dictionary(df_train, unique_question_ids_train, num_questions_train, num_answers)
q_a_dictionary_dev   = q_a_dictionary(df_dev,   unique_question_ids_dev,   num_questions_dev,   num_answers)
q_a_dictionary_test  = q_a_dictionary(df_test,  unique_question_ids_test,   num_questions_test,  num_answers)

print("Time: %0.2f seconds" % (time.time()-start))


Time: 593.88 seconds
Time: 414.02 seconds
Time: 18.05 seconds
Time: 18.23 seconds
Time: 1044.18 seconds


In [13]:
# Testing
print(unique_question_ids_dev)
print(q_a_dictionary_dev[55671683  ])
df_dev[df_dev.question_id == 55671683 ]

[55671683 55670625 55755381 ... 50124645 49918167 50153800]
[(55672089, 1), (40324245, 0), (57235257, 0), (64405028, 0), (40815275, 0), (57778180, 0), (52599934, 0), (40766588, 0), (47615087, 0), (54163988, 0)]


,question_id,question_title_raw,question_body_raw,question_body,question_body_text,question_body_code,question_title_body_raw,question_title_body,question_title_body_text,question_title_body_code,answer_id,answer_body_raw,answer_body,answer_text,answer_code,answer_count,view_count,answer_score,is_accepted_answer
0,55671683,Getting data correctly from SPAN ID beatifulsoup,<p>I need to bring a specific value of the pag...,I need to bring a specific value of the page a...,I need to bring a specific value of the page a...,import urllib.request\nfrom bs4 import Beautif...,Getting data correctly from SPAN ID beatifulso...,I need to bring a specific value of the page a...,I need to bring a specific value of the page a...,import urllib.request\nfrom bs4 import Beautif...,55672089,<p>Change the URL as follows:</p>\n\n<pre><cod...,Change the URL as follows:\nimport requests\nf...,Change the URL as follows:\n,import requests\nfrom bs4 import BeautifulSoup...,1,21,0,1


## Save to Pickle Files

In [14]:
# Save q_a_dictionaries to file
q_a_dict_all_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_all.pickle', 'wb')
pickle.dump(q_a_dictionary_all, q_a_dict_all_to_file)

q_a_dict_train_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_train.pickle', 'wb')
pickle.dump(q_a_dictionary_train, q_a_dict_train_to_file)

q_a_dict_dev_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_dev.pickle', 'wb')
pickle.dump(q_a_dictionary_dev, q_a_dict_dev_to_file)

q_a_dict_test_to_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_test.pickle', 'wb')
pickle.dump(q_a_dictionary_test, q_a_dict_test_to_file)

## Load Dictionaries from Pickle Files

In [15]:
# Load q_a_dictionaries from file
all_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_all.pickle', 'rb')
all_qa_dict = pickle.load(all_from_file)

train_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_train.pickle', 'rb')
train_qa_dict = pickle.load(train_from_file)

dev_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_dev.pickle', 'rb')
dev_qa_dict = pickle.load(dev_from_file)

test_from_file = open('/content/drive/MyDrive/Colab Notebooks/W266 Final Project/data/q_a_dict_test.pickle', 'rb')
test_qa_dict = pickle.load(test_from_file)

In [16]:
print(len(all_qa_dict.keys()))
print(len(train_qa_dict.keys()))
print(len(dev_qa_dict.keys()))
print(len(test_qa_dict.keys()))

30623
24498
3062
3063


In [17]:
for each in test_qa_dict.keys():
  print(test_qa_dict[each])

[(55662134, 0), (55662053, 0), (55661981, 1), (64717925, 0), (52250747, 0), (47805233, 0), (60398311, 0), (47860893, 0), (50565510, 0), (54576739, 0)]
[(55664726, 0), (55664585, 1), (55664663, 0), (51611111, 0), (55118801, 0), (61665527, 0), (58582708, 0), (59407189, 0), (61027439, 0), (42335581, 0)]
[(55665839, 0), (55644071, 0), (55644063, 1), (43736281, 0), (56789874, 0), (61598381, 0), (38446230, 0), (57796365, 0), (60590369, 0), (37491434, 0)]
[(55669044, 0), (55668752, 1), (40313673, 0), (61019118, 0), (42076860, 0), (46987965, 0), (42858938, 0), (55284252, 0), (62841773, 0), (42798604, 0)]
[(55708016, 1), (62266543, 0), (41499488, 0), (51143548, 0), (35772065, 0), (50058945, 0), (38295966, 0), (46439272, 0), (56278456, 0), (63433599, 0)]
[(55750129, 0), (55742051, 1), (62604102, 0), (49949152, 0), (61946349, 0), (52242328, 0), (62702440, 0), (55365732, 0), (57829952, 0), (50031299, 0)]
[(55780627, 1), (61637526, 0), (61555531, 0), (61811276, 0), (51837554, 0), (45779430, 0), (45

In [18]:
for each in train_qa_dict.keys():
  print(train_qa_dict[each])

Streaming output truncated to the last 5000 lines.
[(56948361, 0), (56948422, 0), (56948354, 1), (64960224, 0), (62936853, 0), (46900548, 0), (49934793, 0), (46789564, 0), (58892814, 0), (62111867, 0)]
[(56948769, 0), (56948922, 0), (56948742, 1), (41024664, 0), (61239255, 0), (45641657, 0), (35094803, 0), (41201644, 0), (51731526, 0), (62206474, 0)]
[(56950245, 1), (63835934, 0), (39238276, 0), (51097491, 0), (56497733, 0), (64756816, 0), (40235290, 0), (39637230, 0), (55131241, 0), (39964417, 0)]
[(56951690, 0), (56951763, 0), (56951372, 1), (61845142, 0), (59002277, 0), (53132856, 0), (62839353, 0), (53347772, 0), (54673756, 0), (54121178, 0)]
[(56952207, 0), (56952335, 0), (56952289, 1), (60895227, 0), (57912039, 0), (36069178, 0), (50495635, 0), (60420458, 0), (58847997, 0), (46945157, 0)]
[(56956569, 0), (56952528, 0), (56952437, 1), (59866424, 0), (48234725, 0), (51186177, 0), (46286975, 0), (62429311, 0), (64586315, 0), (65105879, 0)]
[(56960433, 1), (60678843, 0), (55938967, 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

